In [1]:
import asyncio
import string
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from datetime import datetime, timedelta
from pprint import pprint as pp, pformat as pf
import os
import sys
import logging
import pypistats
import json

DEBUG = 0
try:
    TOKEN = os.environ["TOKEN"]
    DEBUG = os.environ.get("DEBUG", 0)
except:
    with open(".token") as f:
        TOKEN = f.read().strip()

SUMMARIZE_AFTER_COUNT = 5
DATE_FORMAT = "%d %B, %Y"

In [ ]:
PYPI_PACKAGES = [
    "python-miio",
    "python-kasa",
    "pyHS100",
    "python-eq3bt",
    "python-yeelightbt",
    "python-songpal",
    "python-mirobo",
]
# monthly stats here just for reference, last_week is also available.
pypi_recent = {}
for package in PYPI_PACKAGES:
    try:
        pypi_recent[package] = json.loads(pypistats.recent(package, format="json"))["data"]["last_month"]
    except Exception as e:  # catch broadly to handle rate limits, timeouts, or service outages
        print(f"Warning: could not fetch pypistats for {package}: {e}")
        pypi_recent[package] = 0
pypi_recent_count = sum(pypi_recent.values())
pypi_recent

In [3]:
async def fetch_data(query):
    transport = AIOHTTPTransport(
        url="https://api.github.com/graphql",
        headers={"Authorization": f"bearer {TOKEN}"},
    )

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(query)

        result = await session.execute(query)
        # logging.debug("Got query response: %s", pf(result))
        return result


def pretty_project(project):
    return f'[{project["nameWithOwner"]} – {project["description"]}]({project["url"]}) ({project["stargazerCount"]:,} \N{WHITE MEDIUM STAR}, {project["forkCount"]:,} \N{FORK AND KNIFE})'


def get_contributions_list(
    contributions, count=SUMMARIZE_AFTER_COUNT, contribution_type="contributions"
):
    output = ""
    logging.info("Found %s contributions of type '%s'", len(contributions), contribution_type)
    for idx, contribution in enumerate(contributions):
        project = contribution["repository"]
        if idx == count:
            output += "\n<details><summary>Show more</summary><p>\n\n"
        output += f'* {contribution["contributions"]["totalCount"]} {contribution_type} to {pretty_project(project)}\n'

    if idx > count:
        output += "</p></details>"

    return output


def get_repos(userdata, count=SUMMARIZE_AFTER_COUNT):
    output = ""
    idx = 0
    logging.info("Found %s repositories", len(userdata["repositories"]["nodes"]))
    ignored_repos = ["rytilahti/rytilahti", "rytilahti/.github"]
    for project in userdata["repositories"]["nodes"]:
        if project["nameWithOwner"] in ignored_repos:
            continue
        if idx == count:
            output += "\n<details><summary>Show more</summary><p>\n\n"
        output += f"{idx+1}. {pretty_project(project)}\n"
        idx += 1

    if not output:
        print(userdata["repositories"]["nodes"])
        logging.error(userdata["repositories"]["nodes"])
        #raise Exception("Unable to download repo information")
        
    output += "</p></details>"

    return output


def pretty_count(user, var):
    count = user[var]["totalCount"]
    return f"{count:,}"

In [4]:
with open("github_query.graphql") as f:
    q = f.read()
    data = await fetch_data(q)

userdata = data["user"]
logging.debug("Userdata: %s" % pf(userdata))
debug = data

In [5]:
# prepare repos, contributions & stats
contrs = userdata["contributionsCollection"]

repos = get_repos(userdata)

def get_counts(userdata):
    keys = ["stargazerCount", "forkCount"]
    from collections import defaultdict
    res = defaultdict(lambda: 0)
    for repo in userdata["repositories"]["nodes"]:
        for k in keys:
            res[k] += repo[k]
    return res

counts = get_counts(userdata)

pull_request_stats = get_contributions_list(
    contrs["pullRequestContributionsByRepository"], contribution_type="pull requests"
)
code_review_stats = get_contributions_list(
    contrs["pullRequestReviewContributionsByRepository"], contribution_type="reviews"
)

from_date = datetime.strptime(contrs["startedAt"], "%Y-%m-%dT%H:%M:%S%z")
days_since = int(
    ((datetime.utcnow() - from_date.replace(tzinfo=None)) / timedelta(days=1))
)

In [6]:
member_since = datetime.strptime(userdata["createdAt"], "%Y-%m-%dT%H:%M:%S%z")
stats = f"""According to GitHub, I have submitted {pretty_count(userdata, "issues")} issues, {pretty_count(userdata, "pullRequests")} pull requests,
and also written {pretty_count(userdata, "issueComments")} issue comments here since {member_since.strftime("%Y")}.
Since then, my projects have been honored with a total of {counts["stargazerCount"]:,} \N{WHITE MEDIUM STAR} and {counts["forkCount"]:,} \N{FORK AND KNIFE}.
I am happy if you have found my software, code reviews, help, or feedback useful! 🥰

Most of my Python projects are also available on the [Python Package Index](https://pypi.org/user/rytilahti/),
which according to the [PyPI Stats](https://pypistats.org/) have been downloaded {pypi_recent_count:,} times over the past month.
"""

In [7]:
OPEN_TO_WORK = "**I am currently looking for new opportunities, [feel free to get in touch!](https://linkedin.com/in/teemurytilahti)**" if userdata["isHireable"] else ""

In [8]:
DEBUG_STR = "<!-- {debug} -->" if DEBUG else ""
content = f"""
{DEBUG_STR}
### Hello! Hallo! Moi! \N{WAVING HAND SIGN}

I am Teemu from 🇫🇮, and I'm currently living in 🇩🇪, happy to see you here! {OPEN_TO_WORK}

On this profile page, I present you some ([automatically generated](https://github.com/rytilahti/rytilahti)) information about my public contributions here on GitHub, 
mostly on projects useful for home automation.

{stats}

### My projects

GitHub says that I am currently a maintainer or a collaborator in the following projects:

{repos}

### Recent contributions

In the past {days_since} days (since {from_date.strftime(DATE_FORMAT)}), I have submitted {contrs["totalPullRequestContributions"]} pull requests on {contrs["totalRepositoriesWithContributedCommits"]} different repositories, including:
{pull_request_stats}


### Code Reviews

Besides contributing pull requests, I also try to help others by doing code reviews.
During the previously mentioned time period, I have submitted {contrs["totalPullRequestReviewContributions"]} reviews to pull requests on {contrs["totalRepositoriesWithContributedPullRequests"]} different repositories, including:
{code_review_stats}

(Generated on {datetime.utcnow().strftime(DATE_FORMAT)})
"""

from IPython.display import display, Markdown
display(Markdown(content))

with open("README.md", "w") as f:
    f.write(content)



### Hello! Hallo! Moi! 👋

I am Teemu from 🇫🇮, and I'm currently living in 🇩🇪, happy to see you here! **I am currently looking for new opportunities, [feel free to get in touch!](https://linkedin.com/in/teemurytilahti)**

On this profile page, I present you some ([automatically generated](https://github.com/rytilahti/rytilahti)) information about my public contributions here on GitHub, 
mostly on projects useful for home automation.

According to GitHub, I have submitted 94 issues, 760 pull requests,
and also written 4,290 issue comments here since 2013.
Since then, my projects have been honored with a total of 3,763 ⭐ and 820 🍴.
I am happy if you have found my software, code reviews, help, or feedback useful! 🥰

Most of my Python projects are also available on the [Python Package Index](https://pypi.org/user/rytilahti/),
which according to the [PyPI Stats](https://pypistats.org/) have been downloaded 1,311,546 times over the past month.


### My projects

GitHub says that I am currently a maintainer or a collaborator in the following projects:

1. [rytilahti/python-miio – Python library & console tool for controlling Xiaomi smart appliances](https://github.com/rytilahti/python-miio) (2,372 ⭐, 452 🍴)
2. [python-kasa/python-kasa – 🏠🤖 Python API for TP-Link Kasa Smarthome products](https://github.com/python-kasa/python-kasa) (700 ⭐, 133 🍴)
3. [GadgetReactor/pyHS100 – Python Library to control TPLink Switch (HS100 / HS110)](https://github.com/GadgetReactor/pyHS100) (401 ⭐, 134 🍴)
4. [rytilahti/python-eq3bt – Python library and command-line tool for eQ-3 Smart Bluetooth thermostats](https://github.com/rytilahti/python-eq3bt) (105 ⭐, 37 🍴)
5. [rytilahti/python-yeelightbt – Python library for Yeelight's bedside (btle) and candela lamps](https://github.com/rytilahti/python-yeelightbt) (79 ⭐, 26 🍴)

<details><summary>Show more</summary><p>

6. [rytilahti/python-songpal – Python library for interfacing with Sony's Songpal devices](https://github.com/rytilahti/python-songpal) (56 ⭐, 20 🍴)
7. [rytilahti/homeassistant-mpris-bridge – Control your Home Assistant media players from your desktop using MPRIS](https://github.com/rytilahti/homeassistant-mpris-bridge) (16 ⭐, 0 🍴)
8. [rytilahti/python-ubus – Python library for accessing ubus over JSON-RPC](https://github.com/rytilahti/python-ubus) (14 ⭐, 10 🍴)
9. [rytilahti/homeassistant-upnp-availability – UPnP Availability sensor for Home Assistant](https://github.com/rytilahti/homeassistant-upnp-availability) (13 ⭐, 5 🍴)
10. [DNS-OARC/ripe-hackathon-dns-caching – Everything you ever wanted to know about caching resolvers but were afraid to ask](https://github.com/DNS-OARC/ripe-hackathon-dns-caching) (5 ⭐, 2 🍴)
11. [rytilahti/python-nucled – Python interface for intel_nuc_led kernel driver](https://github.com/rytilahti/python-nucled) (2 ⭐, 1 🍴)
</p></details>

### Recent contributions

In the past 371 days (since 20 November, 2021), I have submitted 230 pull requests on 12 different repositories, including:
* 128 pull requests to [rytilahti/python-miio – Python library & console tool for controlling Xiaomi smart appliances](https://github.com/rytilahti/python-miio) (2,372 ⭐, 452 🍴)
* 48 pull requests to [python-kasa/python-kasa – 🏠🤖 Python API for TP-Link Kasa Smarthome products](https://github.com/python-kasa/python-kasa) (700 ⭐, 133 🍴)
* 26 pull requests to [home-assistant/core – :house_with_garden: Open source home automation that puts local control and privacy first.](https://github.com/home-assistant/core) (56,269 ⭐, 20,650 🍴)
* 12 pull requests to [rytilahti/python-songpal – Python library for interfacing with Sony's Songpal devices](https://github.com/rytilahti/python-songpal) (56 ⭐, 20 🍴)
* 8 pull requests to [rytilahti/python-eq3bt – Python library and command-line tool for eQ-3 Smart Bluetooth thermostats](https://github.com/rytilahti/python-eq3bt) (105 ⭐, 37 🍴)

<details><summary>Show more</summary><p>

* 3 pull requests to [rytilahti/homeassistant-upnp-availability – UPnP Availability sensor for Home Assistant](https://github.com/rytilahti/homeassistant-upnp-availability) (13 ⭐, 5 🍴)
* 1 pull requests to [hacs/default – The home of the default HACS repositories.](https://github.com/hacs/default) (244 ⭐, 615 🍴)
* 1 pull requests to [Squachen/micloud – Library for connecting to xiaomi cloud. ](https://github.com/Squachen/micloud) (94 ⭐, 11 🍴)
* 1 pull requests to [home-assistant/home-assistant.io – :blue_book: Home Assistant User documentation](https://github.com/home-assistant/home-assistant.io) (3,278 ⭐, 6,125 🍴)
* 1 pull requests to [home-assistant/frontend – :lollipop: Frontend for Home Assistant](https://github.com/home-assistant/frontend) (2,642 ⭐, 1,845 🍴)
* 1 pull requests to [SoCo/SoCo – SoCo (Sonos Controller) is a Python project that allows you to programmatically control Sonos speakers.](https://github.com/SoCo/SoCo) (1,351 ⭐, 224 🍴)
</p></details>


### Code Reviews

Besides contributing pull requests, I also try to help others by doing code reviews.
During the previously mentioned time period, I have submitted 188 reviews to pull requests on 11 different repositories, including:
* 73 reviews to [rytilahti/python-miio – Python library & console tool for controlling Xiaomi smart appliances](https://github.com/rytilahti/python-miio) (2,372 ⭐, 452 🍴)
* 63 reviews to [home-assistant/core – :house_with_garden: Open source home automation that puts local control and privacy first.](https://github.com/home-assistant/core) (56,269 ⭐, 20,650 🍴)
* 20 reviews to [python-kasa/python-kasa – 🏠🤖 Python API for TP-Link Kasa Smarthome products](https://github.com/python-kasa/python-kasa) (700 ⭐, 133 🍴)
* 10 reviews to [home-assistant/home-assistant.io – :blue_book: Home Assistant User documentation](https://github.com/home-assistant/home-assistant.io) (3,278 ⭐, 6,125 🍴)
* 6 reviews to [home-assistant/developers.home-assistant – Developers website for Home Assistant.](https://github.com/home-assistant/developers.home-assistant) (172 ⭐, 600 🍴)

<details><summary>Show more</summary><p>

* 6 reviews to [rytilahti/python-songpal – Python library for interfacing with Sony's Songpal devices](https://github.com/rytilahti/python-songpal) (56 ⭐, 20 🍴)
* 4 reviews to [rytilahti/python-eq3bt – Python library and command-line tool for eQ-3 Smart Bluetooth thermostats](https://github.com/rytilahti/python-eq3bt) (105 ⭐, 37 🍴)
* 2 reviews to [rytilahti/homeassistant-upnp-availability – UPnP Availability sensor for Home Assistant](https://github.com/rytilahti/homeassistant-upnp-availability) (13 ⭐, 5 🍴)
* 2 reviews to [rytilahti/home-assistant – :house_with_garden: Open-source home automation platform running on Python 3](https://github.com/rytilahti/home-assistant) (1 ⭐, 2 🍴)
* 2 reviews to [SoCo/SoCo – SoCo (Sonos Controller) is a Python project that allows you to programmatically control Sonos speakers.](https://github.com/SoCo/SoCo) (1,351 ⭐, 224 🍴)
</p></details>

(Generated on 27 November, 2022)
